In [1]:
#import libraries
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
assert tf.executing_eagerly()

import tensorflow_hub as hub
import numpy as np

import librosa

In [2]:
# function to convert audio signal to numpy array
def convert_audio_to_nd(audio_file):

    y, sr = librosa.load(audio_file,sr = 16000)
    print("Audio data type:", type(y))
    print("Audio data shape:", y.shape)

    
    audio_data = np.array(y)
    print("New audio data type:", type(audio_data))
    print("New audio data shape:", audio_data.shape)


    return audio_data,sr

In [3]:
# function to convert audio signal numpy array to embeddings
def get_audio_embeddings(audio_signal,sr):
    # Load the module and run inference.
    module = hub.load('https://tfhub.dev/google/nonsemantic-speech-benchmark/trill/2')
    
    # Reshape the input ndarray to have shape (num_samples,).
    audio_signal = np.squeeze(audio_signal)
    
    # Resample the audio signal to 16kHz, if necessary.
    # resampler = tf.signal.resample
    # audio_signal = resampler(audio_signal, tf.constant([len(audio_signal) * 16000 // len(audio_signal)], dtype=tf.int32))
    
    #audio_signal = librosa.resample(audio_signal, sr, 16000)

    # Normalize the audio signal to have values between -1 and 1.
    audio_signal = np.asarray(audio_signal, dtype=np.float32)
    assert audio_signal.ndim == 1
    assert np.abs(audio_signal).max() <= 1.
    audio_signal = np.clip(audio_signal, -1., 1.)
    
    # Generate the embeddings using the loaded module.
    emb_dict = module(samples=audio_signal, sample_rate=16000)
    emb = emb_dict['embedding']
    emb_layer19 = emb_dict['layer19']
    
    # Return the embeddings as numpy ndarrays.
    return emb.numpy(), emb_layer19.numpy()



In [4]:
# function to calculate FAD from test & reference embeddings
def calc_fad(ref_emb,test_emb):

    euclidean_sq = np.sum((ref_emb.mean(axis=0) - test_emb.mean(axis=0))**2)

  
    ref_var = np.var(ref_emb, axis=0)
    test_var = np.var(test_emb, axis=0)
    euclidean_var = np.sum(ref_var + test_var - 2*np.sqrt(ref_var*test_var))
    euclidean_var /= ref_emb.shape[1]

    # Compute the Fréchet Audio Distance using the Euclidean distance.
    fad = euclidean_sq + euclidean_var
    fad = np.sqrt(fad)
    return fad


In [5]:
# get reference & test audio, convert to embeddings and calculate FAD
test_audio,sr1 = convert_audio_to_nd("/content/sample-0.mp3") # modify this to take generated audio
ref_audio,sr2 = convert_audio_to_nd("/content/real.mp3") # modify this to take corrosponsing music caps audio

test_emb,test_emb_19 = get_audio_embeddings(test_audio,sr1)
ref_emb,ref_emb_19 = get_audio_embeddings(ref_audio,sr2)

fad = calc_fad(test_emb,ref_emb)
print(fad)

Audio data type: <class 'numpy.ndarray'>
Audio data shape: (160683,)
New audio data type: <class 'numpy.ndarray'>
New audio data shape: (160683,)
Audio data type: <class 'numpy.ndarray'>
Audio data shape: (95109,)
New audio data type: <class 'numpy.ndarray'>
New audio data shape: (95109,)
0.5838273918180972
